In [3]:
import os 
import sys 

cur_path = os.path.abspath(".")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.pagination import ShallowStrategy

from utils.utils import remove_prefix, ddf 

load_dotenv()

True

In [4]:
sg = Subgrounds()
bs: Subgraph = sg.load_subgraph(os.environ['SUBGRAPH_URL'])

# Fertilizer Purchased by Humidity 

In [5]:
pool = bs.Query.metapoolOracles(first=10000, orderBy="season", orderDirection="asc")
df_pool_raw = sg.query_df(
    [
        pool.balanceA, 
        pool.balanceB, 
        pool.season, 
        pool.deltaB, 
    ],
    pagination_strategy=ShallowStrategy
)

In [6]:
df_pool = df_pool_raw.copy()
df_pool = remove_prefix(df_pool, "metapoolOracles_")
df_pool.balanceA /= 10**6
df_pool.balanceB /= 10**18
df_pool['diff'] = df_pool.balanceB - df_pool.balanceA
print(len(df_pool))
df_pool.tail()

701


,balanceA,balanceB,season,deltaB,diff
696,42180980693283.140625,41747964810874.351562,6772,614605,-433015882408.789062
697,42235473473019.453125,41801287288294.117188,6773,-271567,-434186184725.335938
698,42290152050821.890625,41854792340128.265625,6774,111241558,-435359710693.625
699,42344996881447.101562,41908461414829.171875,6775,300627393,-436535466617.929688
700,42399325739675.726562,41961625578478.664062,6776,300789152,-437700161197.0625


In [8]:
nearest = alt.selection_single(
    nearest=True, encodings=["x"], on="mouseover", empty="none"
)
brush = alt.selection_interval(encodings=['x'])

base = alt.Chart(df_pool)
x = alt.X("humidity:Q")
y = alt.Y("fertilizer:Q")

# Histogram of fertilizer purchased by season 
histogram = base.mark_bar().encode(
    x=x, y=y
)

# Vertical line to indicate nearest histogram bin 
vline = base.mark_rule(color="grey").encode(
    x=x, size=alt.value(1)
).transform_filter(
    nearest
)

# Text label to indicate value nearest histogram bin 
text = histogram.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, y, alt.value(' '))
)

# Time axis which drives scale of histogram 
time_axis = base.mark_bar().encode(
    x=x, y=y
).add_selection(
    brush 
).resolve_scale(
    'independent'
).properties(height=50)

(
    alt.layer(
        histogram.add_selection(nearest), 
        text, 
        vline
    ).transform_filter(
        brush
    ).properties(height=250) 
    & time_axis 
) 

alt.VConcatChart(...)

# Chopping 

In [9]:
ferts = bs.Query.fertilizerBalances(first=10000)

df_ferts_raw = sg.query_df(
    [
        ferts.id, 
        ferts.amount, 
        ferts.farmer, 
        ferts.fertilizerToken, 
    ],
    pagination_strategy=ShallowStrategy
)
#.sort_values('podFills_transaction_timestamp').reset_index(drop=True)

In [10]:
df_ferts = df_ferts_raw.copy()
print(len(df_ferts))
print(df_ferts.fertilizerBalances_amount.sum())
print(df_ferts.fertilizerBalances_amount.sum())
print(df_ferts.columns)
df_ferts.head()

1094
17212909
17212909
Index(['fertilizerBalances_id', 'fertilizerBalances_amount',
       'fertilizerBalances_farmer_id', 'fertilizerBalances_farmer_silo_id',
       'fertilizerBalances_farmer_field_id', 'fertilizerBalances_farmer_sown',
       'fertilizerBalances_fertilizerToken_id',
       'fertilizerBalances_fertilizerToken_fertilizer_id',
       'fertilizerBalances_fertilizerToken_supply',
       'fertilizerBalances_fertilizerToken_humidity',
       'fertilizerBalances_fertilizerToken_endBpf',
       'fertilizerBalances_fertilizerToken_startBpf',
       'fertilizerBalances_fertilizerToken_season'],
      dtype='object')


,fertilizerBalances_id,fertilizerBalances_amount,fertilizerBalances_farmer_id,fertilizerBalances_farmer_silo_id,fertilizerBalances_farmer_field_id,fertilizerBalances_farmer_sown,fertilizerBalances_fertilizerToken_id,fertilizerBalances_fertilizerToken_fertilizer_id,fertilizerBalances_fertilizerToken_supply,fertilizerBalances_fertilizerToken_humidity,fertilizerBalances_fertilizerToken_endBpf,fertilizerBalances_fertilizerToken_startBpf,fertilizerBalances_fertilizerToken_season
0,1334303-0xbd120e919eb05343dba68863f2f8468bd701...,161,0xbd120e919eb05343dba68863f2f8468bd7010163,None,None,False,1334303,0x402c84de2ce49af88f5e2ef3710ff89bfed36cb6,161,20.0,1334303,134303,6624
1,1418854-0x6d4ed8a1599dee2655d51b245f786293e03d...,50,0x6d4ed8a1599dee2655d51b245f786293e03d58f7,None,None,False,1418854,0x402c84de2ce49af88f5e2ef3710ff89bfed36cb6,50,28.5,1418854,133854,6517
2,1553682-0xc9f817ea7aabe604f5677bf9c1339e32ef1b...,10,0xc9f817ea7aabe604f5677bf9c1339e32ef1b90f0,0xc9f817ea7aabe604f5677bf9c1339e32ef1b90f0,None,False,1553682,0x402c84de2ce49af88f5e2ef3710ff89bfed36cb6,10,42.0,1553682,133682,6490
3,1593637-0x971b4a65048319cc9843dfdbdc8aa97ad47e...,260,0x971b4a65048319cc9843dfdbdc8aa97ad47ef19d,0x971b4a65048319cc9843dfdbdc8aa97ad47ef19d,None,False,1593637,0x402c84de2ce49af88f5e2ef3710ff89bfed36cb6,260,46.0,1593637,133637,6482
4,2078193-0x48e39dfd0450601cb29f74cb43fc913a57fc...,844,0x48e39dfd0450601cb29f74cb43fc913a57fca813,None,None,False,2078193,0x402c84de2ce49af88f5e2ef3710ff89bfed36cb6,844,94.5,2078193,133193,6385


In [ ]:
"""
fertilizer balances by season 
"""

df_ferts = df_ferts_raw.copy()
df_ferts = df_ferts.rename(columns={
    'fertilizerBalances_amount': 'amount',
    'fertilizerBalances_farmer_id': 'address', 
    'fertilizerBalances_fertilizerToken_humidity': 'humidity',
    'fertilizerBalances_fertilizerToken_season': 'season' # season fertilizer was minted in 
})
df_ferts = df_ferts[['amount', 'address', 'season', 'humidity']].sort_values("season")

# ddf(df_ferts)